# Sandbox
For playing around during development.

In [1]:
%pylab inline
import pysd
print pysd.__version__
print pysd.__file__
import pandas as pd
import os
os.path.abspath(pysd.__file__)



Populating the interactive namespace from numpy and matplotlib


/Users/houghton/anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


SyntaxError: invalid syntax (builder.py, line 107)

In [47]:
np.ones(3)*NaN

array([ nan,  nan,  nan])

In [2]:
import xarray as xr

In [4]:
coords = {'Second Dimension Subscript': ['Column 1', 'Column 2'],
          'Third Dimension Subscript': ['Depth 1', 'Depth 2'],
          'One Dimensional Subscript': ['Entry 1', 'Entry 2', 'Entry 3']}
coords

{'One Dimensional Subscript': ['Entry 1', 'Entry 2', 'Entry 3'],
 'Second Dimension Subscript': ['Column 1', 'Column 2'],
 'Third Dimension Subscript': ['Depth 1', 'Depth 2']}

In [43]:
a = xr.DataArray(data=np.zeros(map(len, coords.values()))*NaN, coords=coords)

In [71]:
m = np.empty(3)*NaN
m

array([ nan,  nan,  nan])

In [70]:
np.empty(4)

array([ 1.,  2.,  3.,  4.])

In [49]:
a.sel(One Dimensional Subscript='Column 1')

SyntaxError: invalid syntax (<ipython-input-49-f3f8947e1e9d>, line 1)

In [36]:
a.loc[{'One Dimensional Subscript': ['Entry 1']}]

<xarray.DataArray (Second Dimension Subscript: 2, Third Dimension Subscript: 2, One Dimensional Subscript: 1)>
array([[[ 1.],
        [ 2.]],

       [[ 3.],
        [ 4.]]])
Coordinates:
  * Second Dimension Subscript  (Second Dimension Subscript) |S8 'Column 1' ...
  * Third Dimension Subscript   (Third Dimension Subscript) |S7 'Depth 1' ...
  * One Dimensional Subscript   (One Dimensional Subscript) |S7 'Entry 1'

In [ ]:
a.loc[{'One Dimensional Subscript': ['Entry 1']}]

In [18]:
a.loc[:,:,'Entry 1']

<xarray.DataArray (Second Dimension Subscript: 2, Third Dimension Subscript: 2)>
array([[ 1.,  2.],
       [ 0.,  0.]])
Coordinates:
  * Second Dimension Subscript  (Second Dimension Subscript) |S8 'Column 1' ...
  * Third Dimension Subscript   (Third Dimension Subscript) |S7 'Depth 1' ...
    One Dimensional Subscript   |S7 'Entry 1'

In [37]:
a.loc[:,'Entry 1',:]

KeyError: 'Entry 1'

In [16]:
a.loc['Column 1',:,'Entry 1']

<xarray.DataArray (Third Dimension Subscript: 2)>
array([ 0.,  0.])
Coordinates:
    Second Dimension Subscript  |S8 'Column 1'
  * Third Dimension Subscript   (Third Dimension Subscript) |S7 'Depth 1' ...
    One Dimensional Subscript   |S7 'Entry 1'

In [17]:
a.loc['Column 1',:,'Entry 1'] = 1, 2

In [21]:
a.loc[:,:,'Entry 1'] = [1,2],[3,4]

In [22]:
a

<xarray.DataArray (Second Dimension Subscript: 2, Third Dimension Subscript: 2, One Dimensional Subscript: 3)>
array([[[ 1.,  0.,  0.],
        [ 2.,  0.,  0.]],

       [[ 3.,  0.,  0.],
        [ 4.,  0.,  0.]]])
Coordinates:
  * Second Dimension Subscript  (Second Dimension Subscript) |S8 'Column 1' ...
  * Third Dimension Subscript   (Third Dimension Subscript) |S7 'Depth 1' ...
  * One Dimensional Subscript   (One Dimensional Subscript) |S7 'Entry 1' ...

In [25]:
a.loc[:,:,'Entry 1'] = [1,2],[3,4]

In [26]:
a

<xarray.DataArray (Second Dimension Subscript: 2, Third Dimension Subscript: 2, One Dimensional Subscript: 3)>
array([[[ 1.,  0.,  0.],
        [ 2.,  0.,  0.]],

       [[ 3.,  0.,  0.],
        [ 4.,  0.,  0.]]])
Coordinates:
  * Second Dimension Subscript  (Second Dimension Subscript) |S8 'Column 1' ...
  * Third Dimension Subscript   (Third Dimension Subscript) |S7 'Depth 1' ...
  * One Dimensional Subscript   (One Dimensional Subscript) |S7 'Entry 1' ...

In [50]:
m = {'Dim1': ['A', 'B'],
                                      'Dim2': ['C', 'D', 'E'],
                                      'Dim3': ['F', 'G', 'H', 'I']}

In [53]:
m.iterkeys()

<dictionary-keyiterator at 0x10d7459f0>

In [56]:
a = 'hi'
a = a or None
a

'hi'

In [57]:
b = None
b = b or 'hi'
b

'hi'

In [60]:
c = 'hi there'
c = c or b
c

'hi there'

In [61]:
b = None
b = b + 'hi'
b

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [62]:
a = ['hi']
a += ['bob']
a

['hi', 'bob']

In [66]:
 or 'hi'

' '

In [64]:
None + 'hi'  'there'

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'